# CoSTCo

In [1]:
import sys
sys.path.append('../src')
from read import *
from model import *
from train import *
from metric import *
device = 'cuda:0'

In [2]:
data = 'lastfm_time'
data_path = '../data'

cfg = DotMap()
cfg.name = data
cfg.dpath = data_path
cfg.rank = 10
cfg.lr = 0.001
cfg.wd = 0.001
cfg.n_iters = 10000
cfg.device = device
cfg.unfair = 0.05
cfg.random = 1
cfg.bs = 1024

verbose = True

tensor = TensorDataset(cfg=cfg, path=cfg.dpath, name=cfg.name)
cfg.sizes = tensor.sizes

***********************************************************************************
[1] Read lastfm_time self...
[2] Read metadata...
[3] No normalization; values are already binary...
[4] Split the tensor into training/validation/test
 [4 - 1] Sparsify the minority group to make it more unfair
[5] Make statistics of group information
[6] Change the date type into torch
[7] Read lastfm_time tensor done...!
Tensor      || ['user', 'artist', 'time']; value
NNZ         || [861, 3066, 1586]; 76727 | 14311 | 14311
Sens. Attr  || user, gender: maj(['M']) min(['F'])
Entity      || Majority: 493 Minority: 368
NNZ         || Majority: [74740] Minority: [1987]
***********************************************************************************


In [3]:
# cfg.aug_tf = 'costco'
cfg.opath = '../output'
cfg.tf = 'costco'
tensor.load_data()

In [4]:
cfg.nc = 64
model = CoSTCo(cfg).to(cfg.device)
model

CoSTCo(
  (factors): ParameterList(
      (0): Parameter containing: [torch.float32 of size 861x10 (GPU 0)]
      (1): Parameter containing: [torch.float32 of size 3066x10 (GPU 0)]
      (2): Parameter containing: [torch.float32 of size 1586x10 (GPU 0)]
  )
  (conv1): Conv2d(1, 64, kernel_size=(1, 3), stride=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(10, 1), stride=(1, 1))
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (last_act): ReLU()
)

In [5]:
trainer = Trainer(model, tensor, cfg, wandb=None)
trainer.train()

Iters:   1 || training loss: 11.61615	Train RMSE: 0.34184 Valid RMSE: 0.34332	
Iters:   2 || training loss: 8.72497	Train RMSE: 0.34067 Valid RMSE: 0.34212	
Iters:   3 || training loss: 8.66418	Train RMSE: 0.33902 Valid RMSE: 0.34050	
Iters:   4 || training loss: 8.58709	Train RMSE: 0.33744 Valid RMSE: 0.33891	
Iters:   5 || training loss: 8.48847	Train RMSE: 0.33498 Valid RMSE: 0.33645	
Iters:   6 || training loss: 8.34956	Train RMSE: 0.33183 Valid RMSE: 0.33341	
Iters:   7 || training loss: 8.15419	Train RMSE: 0.32816 Valid RMSE: 0.33003	
Iters:   8 || training loss: 7.85200	Train RMSE: 0.31796 Valid RMSE: 0.32087	
Iters:   9 || training loss: 7.33020	Train RMSE: 0.30309 Valid RMSE: 0.30968	
Iters:  10 || training loss: 6.60991	Train RMSE: 0.28275 Valid RMSE: 0.29829	
Iters:  11 || training loss: 5.88734	Train RMSE: 0.26382 Valid RMSE: 0.29271	
Iters:  12 || training loss: 5.22789	Train RMSE: 0.25219 Valid RMSE: 0.28719	
Iters:  13 || training loss: 4.85484	Train RMSE: 0.24409 Valid 

In [6]:
res = evaluate_model(model, tensor)
print(f"MSE : {res['test_rmse'] * res['test_rmse']:.4f}")
print(f"MADE: {abs(res['MAD_Error']):.5f} Error1 : {res['Group0_Error']:.5f} Error2: {res['Group1_Error']:.5f}")

Test NRE: 0.2784 Test RMSE: 0.2583
***********************************************************************************
Calculate group fairness...
***********************************************************************************
MSE : 0.0667
MADE: 0.05433 Error1 : 0.04742 Error2: 0.10175
